# Name - Arnav Samal (NIT Rourkela)

Q1 — Vision Transformer on CIFAR-10 (PyTorch)
Goal. Implement a ViT and train on CIFAR-10 (10 classes). Your objective is to achieve the highest
possible test accuracy. You are free to try various improvements and tricks to improve performance.

- Note: You must only use Colab
- Paper: An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy
et al., ICLR 2021).

Requirements
- Patchify images, add learnable positional embeddings, prepend a CLS token, stack
Transformer encoder blocks (MHSA + MLP with residual + norm), classify from CLS.
- In README.md, include: how to run in Colab, the config for your best model, and a tiny
results table (Overall classification test accuracy).

(Bonus — optional analysis)
Any concise analysis earns bonus marks (e.g., patch size choices, depth/width trade-offs,
augmentation effects, optimizer/schedule variants, overlapping vs. non-overlapping patches, etc.).
Keep it short and crisp. The analysis should also be the part of the README.md

# Imports & Installations

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import math
import copy
import time
import gc
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
config = {
    "img_size": 32,
    "patch_size": 4,
    "in_channels": 3,
    "num_classes": 10,
    "embed_dim": 256,
    "num_heads": 8,
    "mlp_dim": 512,
    "num_layers": 6,
    "dropout": 0.1,
    "lr": 3e-4,
    "batch_size": 512,
    "epochs": 20
}

# Model Building

In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_dim, overlap=0):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = ((img_size - overlap) // (patch_size - overlap)) ** 2
        self.proj = nn.Conv2d(in_channels, embed_dim,
                              kernel_size=patch_size, stride=patch_size - overlap)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim))

    def forward(self, x):
        B = x.size(0)
        x = self.proj(x).flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        return x

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

In [5]:
class ViT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_embed = PatchEmbedding(config["img_size"], config["patch_size"],
                                          config["in_channels"], config["embed_dim"])
        self.transformer = nn.Sequential(
            *[TransformerBlock(config["embed_dim"], config["num_heads"], config["mlp_dim"], config["dropout"])
              for _ in range(config["num_layers"])]
        )
        self.norm = nn.LayerNorm(config["embed_dim"])
        self.head = nn.Linear(config["embed_dim"], config["num_classes"])

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.transformer(x)
        cls_token = self.norm(x[:, 0])
        x = self.head(cls_token)
        return x

# Data Loading

In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=2, pin_memory=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=2, pin_memory=False)

100%|██████████| 170M/170M [00:03<00:00, 53.8MB/s]


In [ ]:
def train(model, trainloader, testloader, config, optimizer=None):
    criterion = nn.CrossEntropyLoss().to(device)
    model.to(device)

    if optimizer is None:
        optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

    for epoch in range(config["epochs"]):
        model.train()
        running_loss = 0.0
        for imgs, labels in tqdm(trainloader):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)  
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            del imgs, labels, outputs, loss
            torch.cuda.empty_cache()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in testloader:
                imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(imgs)
                _, predicted = outputs.max(1)

                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                del imgs, labels, outputs, predicted 
            torch.cuda.empty_cache()

        acc = 100.0 * correct / total
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, Test Acc: {acc:.2f}%")

    return float(acc)

model = ViT(config)
train(model, trainloader, testloader, config)

model.to("cpu") 
del model         
gc.collect()      
torch.cuda.empty_cache() 

100%|██████████| 98/98 [00:38<00:00,  2.57it/s]


Epoch 1, Loss: 1.8573, Test Acc: 38.72%


100%|██████████| 98/98 [00:36<00:00,  2.65it/s]


Epoch 2, Loss: 1.5661, Test Acc: 47.03%


100%|██████████| 98/98 [00:36<00:00,  2.65it/s]


...

Epoch 18, Loss: 0.7378, Test Acc: 73.74%


100%|██████████| 98/98 [00:37<00:00,  2.65it/s]


Epoch 19, Loss: 0.7106, Test Acc: 74.07%


100%|██████████| 98/98 [00:37<00:00,  2.64it/s]


Epoch 20, Loss: 0.6945, Test Acc: 73.50%


# Extra Analysis

In [ ]:
def get_optimizer(model, optimizer_type="AdamW", lr=3e-4):
    if optimizer_type == "AdamW":
        return torch.optim.AdamW(model.parameters(), lr=lr)
    else:  
        return torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
def run_experiment(config, desc="", trainloader_override=None, optimizer_type="AdamW", overlap=0):
    print(f"\n=== Experiment: {desc} ===")

    model = ViT(config).to(device)
    model.patch_embed = PatchEmbedding(
        img_size=config["img_size"],
        patch_size=config["patch_size"],
        in_channels=config["in_channels"],
        embed_dim=config["embed_dim"],
        overlap=overlap
    ).to(device)

    optimizer = get_optimizer(model, optimizer_type, lr=config["lr"])
    loader = trainloader_override if trainloader_override else trainloader
    acc = train(model, loader, testloader, config, optimizer=optimizer)
    model.to("cpu")   
    del optimizer
    del model         
    gc.collect()      
    torch.cuda.empty_cache() 
    print(torch.cuda.memory_summary())
    return acc

In [10]:
config["patch_size"] = 4
config["num_layers"] = 6
config["embed_dim"] = 256
baseline_acc = run_experiment(config, "Baseline: patch=4, layers=6, embed=256")

config["patch_size"] = 8
patch8_acc = run_experiment(config, "Patch size 8x8")

config["num_layers"] = 4
config["patch_size"] = 4
shallow_acc = run_experiment(config, "Shallow: layers=4")

config["embed_dim"] = 512
wide_acc = run_experiment(config, "Wider embedding: 512")

trainset_no_aug = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                               transform=transform_test)
trainloader_no_aug = DataLoader(trainset_no_aug, batch_size=config["batch_size"], shuffle=True)
no_aug_acc = run_experiment(config, "No data augmentation", trainloader_override=trainloader_no_aug)

experiments = [
    {"patch_size":8, "overlap":0, "optimizer":"AdamW", "desc":"Baseline, non-overlapping, AdamW"},
    {"patch_size":8, "overlap":4, "optimizer":"AdamW", "desc":"Overlapping patches, AdamW"},
    {"patch_size":8, "overlap":0, "optimizer":"Adam", "desc":"Non-overlapping, Adam"},
]

results = {}
for exp in experiments:
    config["patch_size"] = exp["patch_size"]
    acc = run_experiment(config, exp["desc"], optimizer_type=exp["optimizer"], overlap=exp["overlap"])
    results[exp["desc"]] = acc


=== Experiment: Baseline: patch=4, layers=6, embed=256 ===


100%|██████████| 98/98 [00:37<00:00,  2.64it/s]


Epoch 1, Loss: 1.8546, Test Acc: 38.10%


100%|██████████| 98/98 [00:37<00:00,  2.65it/s]


Epoch 2, Loss: 1.5657, Test Acc: 46.35%


...

Epoch 18, Loss: 0.9832, Test Acc: 65.14%


100%|██████████| 98/98 [00:13<00:00,  7.41it/s]


Epoch 19, Loss: 0.9721, Test Acc: 65.26%


100%|██████████| 98/98 [00:12<00:00,  7.58it/s]


Epoch 20, Loss: 0.9497, Test Acc: 65.61%
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  16640 KiB |   4230 MiB | 195177 GiB | 195177 GiB |
|       from large pool |  16640 KiB |   4188 MiB | 194144 GiB | 194144 GiB |
|       from small pool |      0 KiB |     62 MiB |   1032 GiB |   1032 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  16640 KiB |   4230 MiB | 195177 GiB | 195177 GiB |
|       from large pool

In [11]:
print("\n----- Summary of All Experiments -----")
summary = {
    "Baseline (4x4, 6 layers, 256 embed)": baseline_acc,
    "Patch size 8x8": patch8_acc,
    "Shallow (layers=4)": shallow_acc,
    "Wider embedding (512)": wide_acc,
    "No data augmentation": no_aug_acc,
}
summary.update(results)

for k, v in summary.items():
    print(f"{k}: {v:.2f}%")


----- Summary of All Experiments -----
Baseline (4x4, 6 layers, 256 embed): 73.38%
Patch size 8x8: 65.40%
Shallow (layers=4): 73.06%
Wider embedding (512): 73.40%
No data augmentation: 63.94%
Baseline, non-overlapping, AdamW: 66.10%
Overlapping patches, AdamW: 72.54%
Non-overlapping, Adam: 65.61%
